Load libraries.  In this case we need the ndex library, which you can get with "pip install ndex"

In [2]:
ndex = __import__("ndex")

In [3]:
import requests
import json
import base64

# Connect to NDEx in order to retrieve a network.

In [4]:
anon_ndex = ndex.client.Ndex("http://public.ndexbio.org")

Here we grab the "NCI Pathway Interaction Database - Final Revision - Extended Binary SIF" from NDEx, using its UUID.  The NdexGraph object is an extension of the networkx objects, see https://networkx.github.io/ for all the things you can do with it

In [5]:
G = ndex.networkn.NdexGraph(server='http://public.ndexbio.org', uuid='c0e70804-d848-11e6-86b1-0ac135e8bacf')

In [6]:
nodeid = G.nodes()[0]
G.node[nodeid]['diffusion_input'] = 1.0
print G.node[nodeid]

{u'diffusion_output_rank': 1645, 'name': u'ITSN2', u'selected': True, u'diffusion_output_heat': 0.0006416480485015229, u'aliases': ['uniprot knowledgebase:O95062', 'uniprot knowledgebase:Q15812', 'uniprot knowledgebase:Q9HAK4', 'uniprot knowledgebase:Q9NXE6', 'uniprot knowledgebase:Q9NYG0', 'uniprot knowledgebase:Q9NZM2', 'uniprot knowledgebase:Q9NZM3', 'uniprot knowledgebase:Q9ULG4'], u'type': u'Protein', u'diffusion_input': 1.0}


Call the Diffusion service

In [7]:
url = 'http://v3.heat-diffusion.cytoscape.io'
payload = G.to_cx()
headers = {'content-type': 'application/json'}
response = requests.post(url, data=json.dumps(payload), headers=headers)

consistency group max: 2


#The 'data' dictionary in the json response is a dictionary with node IDs as keys and, 'rank' and 'heat' as the values. 

See the top 10 most relevant nodes to your query. 

In [12]:
nos = []
R = ndex.networkn.NdexGraph(cx=response.json()['data'])
for n in R.nodes_iter():
    if R.node[n]['diffusion_output_rank'] < 10:
        R.node['nid'] = n
        nos.append(R.node[n])
nos = sorted(nos, key=lambda k: k['diffusion_output_rank'])
for no in nos:
    print no
    print "id: " + str(no) + " name: " + no['name'] + " rank: " + str(no['diffusion_output_rank']) + " heat: " + str(no['diffusion_output_heat'])


{u'diffusion_output_rank': 0, 'name': u'ITSN2', u'selected': True, u'diffusion_output_heat': 0.906123911644, u'aliases': "['uniprot knowledgebase:O95062', 'uniprot knowledgebase:Q15812', 'uniprot knowledgebase:Q9HAK4', 'uniprot knowledgebase:Q9NXE6', 'uniprot knowledgebase:Q9NYG0', 'uniprot knowledgebase:Q9NZM2', 'uniprot knowledgebase:Q9NZM3', 'uniprot knowledgebase:Q9ULG4']", u'type': 'Protein', u'diffusion_input': 1.0}
{u'diffusion_output_rank': 1, 'name': u'BRAF', u'selected': True, u'diffusion_output_heat': 0.06318075277, u'aliases': "['uniprot knowledgebase:A4D1T4', 'uniprot knowledgebase:B6HY61', 'uniprot knowledgebase:B6HY62', 'uniprot knowledgebase:B6HY63', 'uniprot knowledgebase:B6HY64', 'uniprot knowledgebase:B6HY65', 'uniprot knowledgebase:B6HY66', 'uniprot knowledgebase:P15056', 'uniprot knowledgebase:Q13878', 'uniprot knowledgebase:Q3MIN6', 'uniprot knowledgebase:Q9UDP8', 'uniprot knowledgebase:Q9Y6T3']", u'type': 'Protein', u'diffusion_input': 1.0}
{u'diffusion_output_ra